# Text generation with Markov chains

Adapted from a notebook by [Allison Parrish](http://www.decontextualize.com/)

In our last workshop, we looked at programs that use simple if-then rules to string together sentences. This is not the only way to make text. Another one calculates the statistical distribution of characters in a text. This is also the basis for modern machine learning, but before we look at this, we will learn about a much older process: Markov chains.

Markov chains are a simple example of *predictive text generation*. It is a method of text generation that make use of a statistical model that, given a certain stretch of text, *predicts* which bit of text should come next, based on probabilities learned from an existing corpus of text. It was first discussed by Russian mathematician Andrey Markov in 1906, but became an important basis for information theory, particularly that of Claude Shannon (more about him next week). For the moment, it is enough to understand that statistic modeling of text *only looks at the statistical distribution of characters or words, without any regard to their meaning*. However, as the Markov chain shows, the text that is produces without such meaning quickly becomes natural-sounding nevertheless. 

Of course, Markov chains are incredibly primitive compared to neural networks. But they allow one to develop an intuition for one of the most powerful ideas of modern natural language generation: That language can be synthesized without any regard to meaning, simply by using statistics. 

## Working with text files

Before we get started, we'll first need some text! Grab two [plain text files from Project Gutenberg](http://www.gutenberg.org/) (or from another source of your choice) and save them to the same directory as this notebook. (If you get to this notebook through the link on bCourses, the files should already be in your root folder.) The code in the following cell loads into Python variables the contents of *two plain text files*, assigned to variables `text_a` and `text_b`. You'll need to replace the filenames with the names of the files that you downloaded, keeping the quotation marks (`"`) intact. Or you can just use the ones that I provided and change nothing.

In [24]:
text_a = open("1342-0.txt").read()
text_b = open("84-0.txt").read()

These variables are *strings*, which are essentially just long lists of the characters that occur in the text, in the order that they occur. The code in the following cell shows the first two hundred characters of text A:

In [2]:
print(text_a[:200])

﻿The Project Gutenberg EBook of Pride and Prejudice, by Jane Austen

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away 


You can change `text_a` to `text_b` to see the output from your second text, or change `200` to a number of your choosing.

The `random.sample()` function gives us a random sampling of the contents of a variable (as long as that variable is a sequence of things, like a string or a list). So, for example, to see twenty random characters from text B:

In [3]:
import random
random.sample(text_b, 20)

['t',
 'd',
 'i',
 'h',
 ' ',
 's',
 's',
 'r',
 'e',
 'r',
 'i',
 'r',
 'd',
 's',
 'g',
 'v',
 'u',
 'd',
 'i',
 't']

This isn't incredibly helpful on its own, but you'll notice that the characters it drew (probably) more or less follow the expected letter distribution for English (i.e., lots of `e`s and `n`s and `t`s).

Perhaps more interesting would be to see a randomly-sampled list of *words*. To do this, we'll make separate variables for the words in the text, using a Python function called `.split()`, which takes a string and turns it into a list of words contained in that string. The following cell makes two new variables that contain the words from both texts respectively:

In [4]:
a_words = text_a.split()
b_words = text_b.split()

Now, ten random words from both text A and text B:

In [5]:
random.sample(a_words, 10)

['listened',
 'a',
 'see',
 'absence',
 'listening',
 'she',
 'as',
 'exhibit.”',
 'Fitzwilliam,',
 'state']

In [6]:
random.sample(b_words, 10)

['saw',
 'shepherd.',
 'only',
 'rustic,',
 'being',
 'resolved',
 'near',
 'would',
 'feel',
 'with']

The code in the following cell uses Python's `Counter` object to count the *most common* letters in the first of these texts:

In [7]:
from collections import Counter
Counter(text_a).most_common(12)

[(' ', 113941),
 ('e', 70344),
 ('t', 47283),
 ('a', 42156),
 ('o', 41138),
 ('n', 38430),
 ('i', 36273),
 ('h', 33883),
 ('r', 33293),
 ('s', 33292),
 ('d', 22247),
 ('l', 21282)]

Specifying the `a_words` variable gives the most frequent *words* instead:

In [8]:
Counter(a_words).most_common(12)

[('the', 4205),
 ('to', 4121),
 ('of', 3662),
 ('and', 3309),
 ('a', 1945),
 ('her', 1858),
 ('in', 1813),
 ('was', 1795),
 ('I', 1740),
 ('that', 1419),
 ('not', 1356),
 ('she', 1306)]

Compare these to the most common words in text B:

In [9]:
Counter(b_words).most_common(12)

[('the', 4056),
 ('and', 2971),
 ('of', 2741),
 ('I', 2719),
 ('to', 2142),
 ('my', 1631),
 ('a', 1394),
 ('in', 1125),
 ('was', 993),
 ('that', 987),
 ('with', 700),
 ('had', 679)]

## Markov models

The question a Markov chain tries to answer is this: Given a stretch of text (say a string of characters, or run of words), what is most the most likely bit of text to come *next*?

One way to answer this question is with an *n-gram* based Markov model. What's an n-gram? I'm glad you asked!

### N-grams

An n-gram is simply a sequence of units drawn from a longer sequence; in the case of text, the unit in question is usually a *character* or a *word*. For convenience, we'll call the unit of the n-gram is called its level; the length of the n-gram is called its order. For example, the following is a list of all unique character-level order-2 n-grams in the word ´condescendences´:

    co
    on
    nd
    de
    es
    sc
    ce
    en
    nc

And the following is an excerpt from the list of all unique word-level order-5 n-grams in The Road Not Taken:

    Two roads diverged in a
    roads diverged in a yellow
    diverged in a yellow wood,
    in a yellow wood, And
    a yellow wood, And sorry
    yellow wood, And sorry I

N-grams are used frequently in natural language processing and are a basic tool text analysis. Their applications range from programs that correct spelling (since ´the´ is more likely than ´teh´, you can autocorrect according to probablities of character distribution) to creative visualizations to compression algorithms to stylometrics to, like here, generative text.

### What comes next?

A Markov model for text begins with a list of n-grams. But in addition to making this list, we also keep track of what unit of text (word, character, etc.) *follows* each of those n-grams.

Let’s do a quick example by hand. This is the same character-level order-2 n-gram analysis of the (very brief) text “condescendences” as above, but this time keeping track of all characters that follow each n-gram:

| n-grams |	next? |
| ------- | ----- |
|co| n|
|on| d|
|nd| e, e|
|de| s, n|
|es| c, (end of text)|
|sc| e|
|ce| n, s|
|en| d, c|
|nc| e|

From this table, we can determine that while the n-gram `co` is followed by n 100% of the time, and while the n-gram `on` is followed by `d` 100% of the time, the n-gram `de` is followed by `s` 50% of the time, and `n` the rest of the time. Likewise, the n-gram `es` is followed by `c` 50% of the time, and followed by the end of the text the other 50% of the time.

Exercise: Imagine (or even better, write out) what this table might look like if you were analyzing words instead of characters, with a source text of your choice.

### Markov chains: Generating text from a Markov model

The Markov models we created above don't just give us interesting statistical probabilities. It also allows us generate a *new* text with those probabilities by *chaining together predictions*. Here’s how we’ll do it, starting with the order 2 character-level Markov model of `condescendences`: (1) start with the initial n-gram (`co`)—those are the first two characters of our output. (2) Now, look at the last *n* characters of output, where *n* is the order of the n-grams in our table, and find those characters in the “n-grams” column. (3) Choose randomly among the possibilities in the corresponding “next” column, and append that letter to the output. (Sometimes, as with `co`, there’s only one possibility). (4) If you chose “end of text,” then the algorithm is over. Otherwise, repeat the process starting with (2). Here’s a record of the algorithm in action:

    co
    con
    cond
    conde
    conden
    condend
    condendes
    condendesc
    condendesce
    condendesces
    
As you can see, we’ve come up with a word that *looks* like the original word, and could even be passed off as a genuine English word (if you squint at it). *From a statistical standpoint, the output of our algorithm is nearly indistinguishable from the input.* This kind of algorithm—moving from one state to the next, according to a list of probabilities—is known as a Markov chain generator.

### Generating with Markovify

Fortunately, with the invention of digital computers, you don't have to perform this algorithm by hand! In fact, Markov chain text generation has been a pastime of poets and programmers going back [all the way to 1983](https://www.jstor.org/stable/24969024), so it should be no surprise that there are many implementations of the idea in Python that you can download and install. The one we're going to use is [Markovify](https://github.com/jsvine/markovify), a Markov chain text generation library originally developed for BuzzFeed, apparently. It comes with a lot of extra niceties that will make our lives easier, but underneath the hood, it implements an algorithm very similar to the one we just did by hand above.

To install Markovify on your computer, run the cell below:

In [1]:
import sys
!{sys.executable} -m pip install markovify

  Using cached markovify-0.9.4-py3-none-any.whl
  Using cached Unidecode-1.3.8-py3-none-any.whl (235 kB)


And then run this cell to make the library available in your notebook:

In [2]:
import markovify

The code in the following cell creates a new text generator, using the text in the variable specified to build the Markov model, which is then assigned to the variable `generator_a`.

In [4]:
generator_a = markovify.Text(text_a)

You can then call the `.make_sentence()` method to generate a sentence from the model:

In [8]:
print(generator_a.make_sentence())

It was evident that she could reply to the contrary himself.


The `.make_short_sentence()` method allows you to specify a maximum length for the generated sentence:

In [11]:
print(generator_a.make_short_sentence(50))

Were the whole affair.


By default, Markovify tries to generate a sentence that is significantly different from any existing sentence in the input text. As a consequence, sometimes the `.make_sentence()` or `.make_short_sentence()` methods will return `None`, which means that in ten tries it wasn't able to generate such a sentence. You can work around this by increasing the number of times it tries to generate a sufficiently unique sentence using the `tries` parameter:

In [18]:
print(generator_a.make_short_sentence(40, tries=100))

Mr. Gardiner, though unknown.


Or by disabling the check altogether with `test_output=False` (note that this means the generator will occasionally return stretches of text that are present in the source text):

In [23]:
print(generator_a.make_short_sentence(40, test_output=False))

There is nothing to guide us.


### Changing the order

When you create the model, you can specify the order of the model using the `state_size` parameter. It defaults to 2. Let's make two model with different orders and compare:

In [25]:
gen_a_1 = markovify.Text(text_a, state_size=1)
gen_a_4 = markovify.Text(text_a, state_size=4)

In [29]:
print("order 1")
print(gen_a_1.make_sentence(test_output=False))
print()
print("order 4")
print(gen_a_4.make_sentence(test_output=False))

order 1
I have admittance.

order 4
In spite of what her sister must endure.


In general, the higher the order, the more the sentences will seem "coherent" (i.e., more closely resembling the source text). Lower order models will produce more variation. Deciding on the order is usually a matter of taste and trial-and-error.

### Changing the level

Markovify, by default, works with *words* as the individual unit. It doesn't come out-of-the-box with support for character-level models. The following code defines a new kind of Markovify generator that implements character-level models. Execute it before continuing:

In [30]:
class SentencesByChar(markovify.Text):
    def word_split(self, sentence):
        return list(sentence)
    def word_join(self, words):
        return "".join(words)

Any of the parameters you passed to `markovify.Text` you can also pass to `SentencesByChar`. The `state_size` parameter still controls the order of the model, but now the n-grams are characters, not words.

The following cell implements a character-level Markov text generator for the word "condescendences":

In [31]:
con_model = SentencesByChar("condescendences", state_size=2)

Execute the cell below to see the output—it'll be a lot like what we implemented by hand earlier!

In [36]:
con_model.make_sentence()

'condescencences'

Of course, you can use a character-level model on any text of your choice. So, for example, the following cell creates a character-level order-7 Markov chain text generator from text A:

In [41]:
gen_a_char = SentencesByChar(text_a, state_size=7)

And the cell below prints out a random sentence from this generator. (The `.replace()` is to get rid of any newline characters in the output.)

In [44]:
print(gen_a_char.make_sentence(test_output=False).replace("\n", " "))

It was very much, I feel it my duty to the ground, determined.


### Combining models

Markovify has a handy feature that allows you to *combine* models, creating a new model that draws on probabilities from both of the source models. You can use this to create hybrid output that mixes the style and content of two (or more!) different source texts. To do this, you need to create the models independently, and then call `.combine()` to combine them.

In [45]:
generator_a = markovify.Text(text_a)
generator_b = markovify.Text(text_b)
combo = markovify.combine([generator_a, generator_b], [0.5, 0.5])

The bit of code `[0.5, 0.5]` controls the "weights" of the models, i.e., how much to emphasize the probabilities of any model. You can change this to suit your tastes. (E.g., if you want mostly text A with but a *soupçon* of text B, you would write `[0.9, 0.1]`. Try it!) 

Then you can create sentences using the combined model:

In [48]:
print(combo.make_sentence())

But I concealed my struggles, and flattered you into the garden and the principal spokesman, and Miss Lucas, whose civility in listening to him how absolutely necessary to interrupt my tranquillity.


### Bringing it all together

I've pre-written some code below to make it easy for you to experiment and produce output from Markovify. Just make adjustments to the values assigned to the variables in the cell below:

In [49]:
# change to "word" for a word-level model
level = "char"
# controls the length of the n-gram
order = 7
# controls the number of lines to output
output_n = 14
# weights between the models; text A first, text B second.
# if you want to completely exclude one model, set its corresponding value to 0
weights = [0.5, 0.5]
# limit sentence output to this number of characters
length_limit = 280

(The lines beginning with `#` are "comments"—they don't do anything, they're just there to explain what's happening in the code.)

After making your changes above, run the cell below to generate text according to your parameters. Repeat as necessary until you get something you really like!

In [50]:
model_cls = markovify.Text if level == "word" else SentencesByChar
gen_a = model_cls(text_a, state_size=order)
gen_b = model_cls(text_b, state_size=order)
gen_combo = markovify.combine([gen_a, gen_b], weights)
for i in range(output_n):
    out = gen_combo.make_short_sentence(length_limit, test_output=False)
    out = out.replace("\n", " ")
    print(out)
    print()

Mr. Darcy was commenced, however, Felix flitted before Monday: as soon communications.

I found herself.

As he was no horsewoman, about it is done, and on which are to her former occupations, for it had been insufficiently that urgent business.

We were some new observe her temper was not on hearing amidst his early a hundred pounds, and was consolation than Jane.

They found that the notice of time, the whole works provide for the work of my condition to this attentions.

I was guiltless, but my though perhaps, I had known as you had any value he put his repeatedly tried to Mr. Bingley had been industriously, what degree of his cousin, why is not the least could do.

The first an angel become more interesting in this agreed to discover whisper than they always been massacred before been a week since my journey.

She received the purport of your countenance.

We possessed much good angel, 28th March, 17—.   So stranger, and Elizabeth passion and again I shall curse upon the being the 

## Generating with non-prose text

Markovify assumes you're feeding it prose, i.e., a text file that can be parsed into sentences by separating on sentence-ending punctuation. But often you're *not* working with text like this. For example, let's generate some sonnets. First, download [this plaintext version of Shakespeare's sonnets](https://raw.githubusercontent.com/aparrish/plaintext-example-files/master/sonnets.txt) and keep it in the same directory as this notebook. We'll define the sonnet-generating task as consisting of (a) training a Markov chain on lines of poetry and then (b) generating a sequence of fourteen lines of poetry. Since the *line* is the unit now and not the *sentence*, we need to use Markovify's `NewlineText` class instead of `Text`:

In [52]:
sonnets_text = open("sonnets.txt").read()
sonnets_model = markovify.NewlineText(sonnets_text, state_size=1)

And then we can generate:

In [54]:
sonnets_model.make_sentence()

"Flatter the summer and do not sick Muse in thy steel bosom's ward,"

And now make a sonnet, sorta:

In [55]:
for i in range(14):
    print(sonnets_model.make_sentence())

As any wrinkle graven there;
While comments of spirit affords,
O! how shall not to stay and idle rank before.
And so much hold,
To find out even by mutual ordering;
And do not with his golden time.
Then need blood;
Lo! thus, by this line some mother.
Hers by the wise world may stain both moon and truth;
And swear against my slumbers should blunter be gone,
So do if thou see'st the thanks, if thou art,
To leave poor infant's discontent;
Full character'd with loss,
For then find,


Doing this with a character-level model is a bit more tricky. I've written code in the cell below that defines a new class, `LinesByCharacter` that works like `NewlineText` but operates character-by-character instead of word-by-word:

In [83]:
class LinesByChar(markovify.NewlineText):
    def word_split(self, sentence):
        return list(sentence)
    def word_join(self, words):
        return "".join(words)

Now we can create a character model with the sonnets, line by line:

In [90]:
sonnets_char_model = LinesByChar(sonnets_text, state_size=4)

And generate new sonnets:

In [91]:
for i in range(14):
    print(sonnets_char_model.make_sentence())

When I soul provided make soul a for they had not Time's rose, hath to my mistrengthen inwards expresence, being friend
From thy fair whom is away,
With which to thee fair fame that, nor drowning?
For stol'n of small making wood repose wherevery bles brief
That would my ruth,
And kingle life rude lack again,
And in me the earse,
Then make world, or rime:
In the hunter inflame hidden usure,
Yours are there a sad die as and thy outlive?
O! let me world's eyes,
When soul's trange is admire
Call'd fruit;
And the to their broke, as many, see, what dote,


### New moods

Character-level Markov chains are especially suitable, in my experience, for generating shorter texts, like individual words or names. Let's generate names of new moods using this technique. First, download [this JSON file of moods](https://raw.githubusercontent.com/dariusk/corpora/9bb62927951f79bec2454f29d71b6e9b28d874b1/data/humans/moods.json) from [Corpora Project](https://github.com/dariusk/corpora/) and save to the same directory as this notebook. (A JSON file is a specific way of structuring data. Think of it as an excel file: it can have lists with several colums and rows.)

Then load the JSON file and grab just the list of words naming moods:

In [99]:
import json
mood_data = json.loads(open("./moods.json").read())
moods = mood_data['content']

The easiest way to use this is to make one big string with the moods joined together with newlines:

In [100]:
moods_text = "\n".join(moods)

Then use `LinesByChar` to make the model:

In [101]:
moods_char_model = LinesByChar(moods_text, state_size=3)

And voila, just based on the character disctribution of the list of existing moods, Markovify creates new, plausible moods:

In [102]:
for i in range(24):
    print(moods_char_model.make_sentence())

mourned
exploiteful
trainepted
trative
critive
kiddy
melanced
attacted
exhauseated
relitted
yieldisbelittened
disty
jubilankful
defendeciated
mellous
emble
hyperky
wholistificult
grievolemned
carefressed
infered
posabothed
senterplearful
expeciated


You can try this with other lists – for instance, try using the file "firstnames.json" instead of "moods.json" to create new and exciting first names!